<a href="https://colab.research.google.com/github/mritunjoychk/CSE440-Project_Dementia-Detection_Summer-22/blob/main/CSE440%20Merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

from sklearn import svm
from sklearn.metrics import accuracy_score


# **Loading Dataset**

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data= pd.read_excel("/content/drive/MyDrive/CSE 440 Project /oasis_longitudinal_demographics.xlsx")

In [ ]:
data.head()

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1986.550000,0.696106,0.883440
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004.479526,0.681062,0.875539
2,OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NaN,23.0,0.5,1678.290000,0.736336,1.045710
3,OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NaN,28.0,0.5,1737.620000,0.713402,1.010000
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1697.911134,0.701236,1.033623


In [ ]:
data.shape

(373, 15)

# **Pre-Processing**

**Dropping Attributes**

In [ ]:
data = data.drop(columns = ['Subject ID','MRI ID','Hand'])

**Duplicate values**

In [ ]:
data.duplicated().sum()

0

**Null Values**

In [ ]:
data.isnull().sum()

Group        0
Visit        0
MR Delay     0
M/F          0
Age          0
EDUC         0
SES         19
MMSE         2
CDR          0
eTIV         0
nWBV         0
ASF          0
dtype: int64

In [ ]:
data['SES'] = data['SES'].fillna(data['SES'].mode()[0])
data['MMSE'] = data['MMSE'].fillna(data['MMSE'].mode()[0])

In [ ]:
data.isnull().sum()

Group       0
Visit       0
MR Delay    0
M/F         0
Age         0
EDUC        0
SES         0
MMSE        0
CDR         0
eTIV        0
nWBV        0
ASF         0
dtype: int64

In [ ]:
data.head()

,Group,Visit,MR Delay,M/F,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,Nondemented,1,0,M,87,14,2.0,27.0,0.0,1986.550000,0.696106,0.883440
1,Nondemented,2,457,M,88,14,2.0,30.0,0.0,2004.479526,0.681062,0.875539
2,Demented,1,0,M,75,12,2.0,23.0,0.5,1678.290000,0.736336,1.045710
3,Demented,2,560,M,76,12,2.0,28.0,0.5,1737.620000,0.713402,1.010000
4,Demented,3,1895,M,80,12,2.0,22.0,0.5,1697.911134,0.701236,1.033623


 **Encoding**

In [ ]:
data.dtypes

Group        object
Visit         int64
MR Delay      int64
M/F          object
Age           int64
EDUC          int64
SES         float64
MMSE        float64
CDR         float64
eTIV        float64
nWBV        float64
ASF         float64
dtype: object

Label Encoding (Group)

In [ ]:
labelencoder = LabelEncoder()
data['Group_cat'] = labelencoder.fit_transform(data['Group'])

In [ ]:
data.head()

,Group,Visit,MR Delay,M/F,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF,Group_cat
0,Nondemented,1,0,M,87,14,2.0,27.0,0.0,1986.550000,0.696106,0.883440,2
1,Nondemented,2,457,M,88,14,2.0,30.0,0.0,2004.479526,0.681062,0.875539,2
2,Demented,1,0,M,75,12,2.0,23.0,0.5,1678.290000,0.736336,1.045710,1
3,Demented,2,560,M,76,12,2.0,28.0,0.5,1737.620000,0.713402,1.010000,1
4,Demented,3,1895,M,80,12,2.0,22.0,0.5,1697.911134,0.701236,1.033623,1


**Target --> Group_cat:**

*   Converted = 0
*   Demented = 1
*   Non-Demented = 2




In [ ]:
data = data.drop(columns = ['Group'])


In [ ]:
data['Group_cat'].describe()

count    373.000000
mean       1.410188
std        0.664461
min        0.000000
25%        1.000000
50%        2.000000
75%        2.000000
max        2.000000
Name: Group_cat, dtype: float64

In [ ]:
data.head()

,Visit,MR Delay,M/F,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF,Group_cat
0,1,0,M,87,14,2.0,27.0,0.0,1986.550000,0.696106,0.883440,2
1,2,457,M,88,14,2.0,30.0,0.0,2004.479526,0.681062,0.875539,2
2,1,0,M,75,12,2.0,23.0,0.5,1678.290000,0.736336,1.045710,1
3,2,560,M,76,12,2.0,28.0,0.5,1737.620000,0.713402,1.010000,1
4,3,1895,M,80,12,2.0,22.0,0.5,1697.911134,0.701236,1.033623,1


One Hot Encoding (M/F)

In [ ]:
data = pd.get_dummies(data=data, drop_first=True, columns=['M/F'])

In [ ]:
data.head()

,Visit,MR Delay,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF,Group_cat,M/F_M
0,1,0,87,14,2.0,27.0,0.0,1986.550000,0.696106,0.883440,2,1
1,2,457,88,14,2.0,30.0,0.0,2004.479526,0.681062,0.875539,2,1
2,1,0,75,12,2.0,23.0,0.5,1678.290000,0.736336,1.045710,1,1
3,2,560,76,12,2.0,28.0,0.5,1737.620000,0.713402,1.010000,1,1
4,3,1895,80,12,2.0,22.0,0.5,1697.911134,0.701236,1.033623,1,1


In [ ]:
data.dtypes

Visit          int64
MR Delay       int64
Age            int64
EDUC           int64
SES          float64
MMSE         float64
CDR          float64
eTIV         float64
nWBV         float64
ASF          float64
Group_cat      int64
M/F_M          uint8
dtype: object

# **Splitting Dataset**

In [ ]:
X = data.drop(['Group_cat'],axis=1)
y = data['Group_cat']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20, stratify=y)

**STANDARDIZATION**

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

# **Base Model Training**

**Logistic Regression**

In [ ]:
# logistic regression object
lr = LogisticRegression()
  
# train the model on train set
lr.fit(X_train, y_train.ravel())
  
predictions = lr.predict(X_test)
  
# print classification report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.18      0.31        11
           1       0.88      1.00      0.94        44
           2       0.93      0.98      0.96        57

    accuracy                           0.91       112
   macro avg       0.94      0.72      0.73       112
weighted avg       0.92      0.91      0.89       112



**SVM**

In [ ]:

model_svm=svm.SVC(kernel='linear')

model_svm.fit(X_train,y_train.ravel())

X_train_pred=model_svm.predict(X_train)

predi_svm=model_svm.predict(X_test)

print(classification_report(y_test, predi_svm))

              precision    recall  f1-score   support

           0       1.00      0.18      0.31        11
           1       0.88      1.00      0.94        44
           2       0.93      0.98      0.96        57

    accuracy                           0.91       112
   macro avg       0.94      0.72      0.73       112
weighted avg       0.92      0.91      0.89       112



**Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# train the model on train set
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train.ravel())
predi_dt = dt.predict(X_test)
  
# print classification report
print(classification_report(y_test, predi_dt))

              precision    recall  f1-score   support

           0       0.33      0.27      0.30        11
           1       0.88      1.00      0.94        44
           2       0.94      0.88      0.91        57

    accuracy                           0.87       112
   macro avg       0.72      0.72      0.72       112
weighted avg       0.86      0.87      0.86       112



**KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7)
 
knn.fit(X_train, y_train.ravel())
 

predi_knn = knn.predict(X_test)  
# print classification report
print(classification_report(y_test, predi_knn))

              precision    recall  f1-score   support

           0       0.50      0.09      0.15        11
           1       0.85      0.91      0.88        44
           2       0.89      0.98      0.93        57

    accuracy                           0.87       112
   macro avg       0.75      0.66      0.66       112
weighted avg       0.84      0.87      0.84       112



**Naive Bayes**

In [ ]:
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train,y_train.ravel())

X_train_pred=nb.predict(X_train)

predi_nb=nb.predict(X_test)

print(classification_report(y_test, predi_nb))

              precision    recall  f1-score   support

           0       0.60      0.27      0.37        11
           1       0.90      0.98      0.93        44
           2       0.93      0.96      0.95        57

    accuracy                           0.90       112
   macro avg       0.81      0.74      0.75       112
weighted avg       0.89      0.90      0.89       112



**Adaboost**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

from sklearn.svm import SVC

from sklearn import metrics
svc=SVC(probability=True, kernel='linear')
# Create adaboost classifer object
abc =AdaBoostClassifier(n_estimators=50, base_estimator=svc,learning_rate=1)

# Train Adaboost Classifer
model_abc = abc.fit(X_train, y_train.ravel())

#Predict the response for test dataset
predi_abc=model_abc.predict(X_test)


print(classification_report(y_test, predi_abc))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.89      0.91      0.90        44
           2       0.85      1.00      0.92        57

    accuracy                           0.87       112
   macro avg       0.58      0.64      0.61       112
weighted avg       0.78      0.87      0.82       112



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **OVER SAMPLING**

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 400)
X_train_os, y_train_os = sm.fit_resample(X_train, y_train.ravel())
  
print('After OverSampling, the shape of train_X: {}'.format(X_train_os.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_os.shape))

print("After OverSampling, counts of label '0': {}".format(sum(y_train_os == 0)))
print("After OverSampling, counts of label '1': {}".format(sum(y_train_os == 1)))
print("After OverSampling, counts of label '2': {}".format(sum(y_train_os == 2)))


After OverSampling, the shape of train_X: (399, 11)
After OverSampling, the shape of train_y: (399,) 

After OverSampling, counts of label '0': 133
After OverSampling, counts of label '1': 133
After OverSampling, counts of label '2': 133


# **MODEL EVALUATION AFTER OVERSAMPLING**

**Logistic Regression(OS)**

In [ ]:
# logistic regression object
lr_os = LogisticRegression()
lr_os.fit(X_train_os, y_train_os.ravel())
predi_os_lr = lr_os.predict(X_test)
  
# print classification report
print(classification_report(y_test, predi_os_lr))

              precision    recall  f1-score   support

           0       0.44      0.36      0.40        11
           1       0.90      1.00      0.95        44
           2       0.94      0.89      0.92        57

    accuracy                           0.88       112
   macro avg       0.76      0.75      0.76       112
weighted avg       0.88      0.88      0.88       112



**KNN(OS)**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_os = KNeighborsClassifier(n_neighbors=7)
 
knn_os.fit(X_train_os, y_train_os.ravel())
 

predi_os_knn = knn_os.predict(X_test)  
# print classification report
print(classification_report(y_test, predi_os_knn))

              precision    recall  f1-score   support

           0       0.24      0.64      0.35        11
           1       0.88      0.82      0.85        44
           2       0.98      0.72      0.83        57

    accuracy                           0.75       112
   macro avg       0.70      0.72      0.68       112
weighted avg       0.87      0.75      0.79       112



**DECISION TREE(OS)**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# train the model on train set
dt_os = DecisionTreeClassifier()
dt_os.fit(X_train_os, y_train_os.ravel())
predi_os_dt = dt_os.predict(X_test)
  
# print classification report
print(classification_report(y_test, predi_os_dt))

              precision    recall  f1-score   support

           0       0.11      0.36      0.17        11
           1       0.87      0.89      0.88        44
           2       0.94      0.51      0.66        57

    accuracy                           0.64       112
   macro avg       0.64      0.59      0.57       112
weighted avg       0.83      0.64      0.70       112



**SVM(OS)**

In [ ]:
svm_os=svm.SVC(kernel='linear')

svm_os.fit(X_train_os,y_train_os.ravel())

X_train_pred=svm_os.predict(X_train_os)

predi_os_svm=svm_os.predict(X_test)

print(classification_report(y_test, predi_os_svm))

              precision    recall  f1-score   support

           0       0.43      0.27      0.33        11
           1       0.88      1.00      0.94        44
           2       0.95      0.91      0.93        57

    accuracy                           0.88       112
   macro avg       0.75      0.73      0.73       112
weighted avg       0.87      0.88      0.87       112



**NAIVE BAYES(OS)**

In [ ]:
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
nb_os = GaussianNB()
nb_os.fit(X_train_os,y_train_os.ravel())

X_train_pred=nb_os.predict(X_train_os)

predi_os_nb=nb_os.predict(X_test)

print(classification_report(y_test, predi_os_nb))

              precision    recall  f1-score   support

           0       0.29      0.36      0.32        11
           1       0.91      0.89      0.90        44
           2       0.93      0.89      0.91        57

    accuracy                           0.84       112
   macro avg       0.71      0.71      0.71       112
weighted avg       0.86      0.84      0.85       112



**ADABOOST(OS)**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

from sklearn.svm import SVC

from sklearn import metrics
svc=SVC(probability=True, kernel='linear')
# Create adaboost classifer object
abc_os =AdaBoostClassifier(n_estimators=50, base_estimator=svc,learning_rate=1)

# Train Adaboost Classifer
model_abc = abc_os.fit(X_train_os, y_train_os.ravel())

#Predict the response for test dataset
predi_os_abc=model_abc.predict(X_test)


print(classification_report(y_test, predi_os_abc))

              precision    recall  f1-score   support

           0       0.20      0.36      0.26        11
           1       0.88      1.00      0.94        44
           2       0.95      0.70      0.81        57

    accuracy                           0.79       112
   macro avg       0.68      0.69      0.67       112
weighted avg       0.85      0.79      0.80       112

